# Phase 2 – YOLOv8 Baseline (Closed-Set Training)

This notebook fine-tunes a **YOLOv8s** model on the MaSTr1325 dataset  
to build a strong **closed-set baseline** for comparison against  
open-vocabulary detectors (OWL-ViT & Grounding DINO).

---

**Pipeline**
1. Verify environment and GPU
2. Create dataset config (`mast.yaml`)
3. Train YOLOv8 baseline
4. Evaluate on Val / Test
5. Save & visualize results


### Environment & GPU Check

In [26]:
from ultralytics import YOLO
import torch
from pathlib import Path

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Torch: 2.10.0.dev20251112+cu128
CUDA available: True
Device: NVIDIA GeForce RTX 5070 Laptop GPU


### Set up Paths

In [27]:
from pathlib import Path

BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
RUNS_DIR = BASE_DIR / "runs"
MODELS_DIR = BASE_DIR / "models"
print(BASE_DIR)

D:\Sem3\CS581B_Robot_Perception\Project\Implementation


In [28]:
# Your project base directory
BASE_DIR = Path("D:/Sem3/CS581B_Robot_Perception/Project/Implementation")

DATA_DIR = BASE_DIR / "data"
ANNOT_DIR = DATA_DIR / "annotations"
IMG_DIR = DATA_DIR / "extracted" / "MaSTr1325_images_512x384"

print("DATA_DIR:", DATA_DIR)
print("ANNOT_DIR:", ANNOT_DIR)
print("Images exist:", IMG_DIR.exists())

DATA_DIR: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data
ANNOT_DIR: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\annotations
Images exist: True


### Create YOLO Dataset Config

In [29]:
yaml_path = DATA_DIR / "mast.yaml"

yaml_text = f"""
path: {DATA_DIR.as_posix()}
train: {DATA_DIR.as_posix()}/images/train
val: {DATA_DIR.as_posix()}/images/val
test: {DATA_DIR.as_posix()}/images/val

nc: 1
names:
  0: obstacle
"""

with open(yaml_path, "w") as f:
    f.write(yaml_text.strip())

print("✔ YAML created at:", yaml_path)
print(yaml_path.read_text())


✔ YAML created at: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\mast.yaml
path: D:/Sem3/CS581B_Robot_Perception/Project/Implementation/data
train: D:/Sem3/CS581B_Robot_Perception/Project/Implementation/data/images/train
val: D:/Sem3/CS581B_Robot_Perception/Project/Implementation/data/images/val
test: D:/Sem3/CS581B_Robot_Perception/Project/Implementation/data/images/val

nc: 1
names:
  0: obstacle


### Convert COCO JSON to YOLO TXT

In [30]:
import json
from tqdm import tqdm

def convert_coco_to_yolo(coco_json, output_dir):
    coco_json = Path(coco_json)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    with open(coco_json, 'r') as f:
        data = json.load(f)

    img_map = {img["id"]: img["file_name"] for img in data["images"]}
    wh_map = {img["id"]: (img["width"], img["height"]) for img in data["images"]}

    for ann in tqdm(data["annotations"], desc=f"Converting {coco_json.name}"):
        img_id = ann["image_id"]
        bbox = ann["bbox"]
        x, y, w, h = bbox
        W, H = wh_map[img_id]

        x_center = (x + w/2) / W
        y_center = (y + h/2) / H
        w /= W
        h /= H

        txt_path = output_dir / Path(img_map[img_id]).with_suffix(".txt")
        with open(txt_path, "a") as f_out:
            f_out.write(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print(f"✔ {coco_json.name}: {len(data['annotations'])} boxes converted")

# Run conversions
convert_coco_to_yolo(ANNOT_DIR/"mast_COCO_train.json", DATA_DIR/"labels"/"train")
convert_coco_to_yolo(ANNOT_DIR/"mast_COCO_val.json", DATA_DIR/"labels"/"val")


Converting mast_COCO_train.json: 100%|██████████| 2303/2303 [00:00<00:00, 3941.85it/s]


✔ mast_COCO_train.json: 2303 boxes converted


Converting mast_COCO_val.json: 100%|██████████| 691/691 [00:00<00:00, 3813.36it/s]

✔ mast_COCO_val.json: 691 boxes converted


### Organize Train/Val Images

In [31]:
import shutil
from sklearn.model_selection import train_test_split

SRC_IMAGES = DATA_DIR/"extracted"/"MaSTr1325_images_512x384"

TRAIN_DIR = DATA_DIR/"images"/"train"
VAL_DIR   = DATA_DIR/"images"/"val"

TRAIN_DIR.mkdir(parents=True, exist_ok=True)
VAL_DIR.mkdir(parents=True, exist_ok=True)

all_imgs = sorted(list(SRC_IMAGES.glob("*.jpg")))

train_imgs, val_imgs = train_test_split(all_imgs, test_size=0.2, random_state=42)

def copy_images(img_list, dest):
    for p in img_list:
        shutil.copy(p, dest / p.name)

print("Copying train images...")
copy_images(train_imgs, TRAIN_DIR)

print("Copying val images...")
copy_images(val_imgs, VAL_DIR)

print(f"✔ Train: {len(train_imgs)}, Val: {len(val_imgs)}")


Copying train images...
Copying val images...
✔ Train: 1060, Val: 265


### Clean Labels(Remove invalid or mismatched one)

In [32]:
label_dirs = [DATA_DIR/"labels"/"train", DATA_DIR/"labels"/"val"]

def clean_labels(label_dir, img_dir):
    img_stems = {p.stem for p in img_dir.glob("*.jpg")}
    for txt in label_dir.glob("*.txt"):
        if txt.stem not in img_stems:
            txt.unlink()
            print("🗑 Removed orphan label:", txt.name)

clean_labels(DATA_DIR/"labels"/"train", TRAIN_DIR)
clean_labels(DATA_DIR/"labels"/"val", VAL_DIR)

print("✔ Label cleaning completed")

🗑 Removed orphan label: 0032.txt
🗑 Removed orphan label: 0044.txt
🗑 Removed orphan label: 0045.txt
🗑 Removed orphan label: 0049.txt
🗑 Removed orphan label: 0050.txt
🗑 Removed orphan label: 0052.txt
🗑 Removed orphan label: 0055.txt
🗑 Removed orphan label: 0057.txt
🗑 Removed orphan label: 0064.txt
🗑 Removed orphan label: 0079.txt
🗑 Removed orphan label: 0087.txt
🗑 Removed orphan label: 0102.txt
🗑 Removed orphan label: 0108.txt
🗑 Removed orphan label: 0110.txt
🗑 Removed orphan label: 0129.txt
🗑 Removed orphan label: 0169.txt
🗑 Removed orphan label: 0177.txt
🗑 Removed orphan label: 0182.txt
🗑 Removed orphan label: 0188.txt
🗑 Removed orphan label: 0189.txt
🗑 Removed orphan label: 0198.txt
🗑 Removed orphan label: 0206.txt
🗑 Removed orphan label: 0210.txt
🗑 Removed orphan label: 0222.txt
🗑 Removed orphan label: 0224.txt
🗑 Removed orphan label: 0232.txt
🗑 Removed orphan label: 0234.txt
🗑 Removed orphan label: 0235.txt
🗑 Removed orphan label: 0247.txt
🗑 Removed orphan label: 0251.txt
🗑 Removed 

In [33]:
from pathlib import Path

def fix_corrupt_labels(label_dir):
    for txt in Path(label_dir).glob("*.txt"):
        lines = txt.read_text().strip().splitlines()
        cleaned = []
        changed = False

        for ln in lines:
            parts = ln.split()
            if not parts:
                continue

            cls = int(parts[0])

            # Fix invalid class IDs (dataset has only class 0)
            if cls != 0:
                parts[0] = "0"
                changed = True

            cleaned.append(" ".join(parts))

        # Remove duplicate lines
        uniq = list(dict.fromkeys(cleaned))
        if len(uniq) != len(cleaned):
            changed = True

        if changed:
            print("🔧 Fixed:", txt.name)
            txt.write_text("\n".join(uniq))

# Run fixes for train and val
fix_corrupt_labels(DATA_DIR/"labels"/"train")
fix_corrupt_labels(DATA_DIR/"labels"/"val")

print("✔ Corrupt label fixing completed")


🔧 Fixed: 0001.txt
🔧 Fixed: 0003.txt
🔧 Fixed: 0005.txt
🔧 Fixed: 0006.txt
🔧 Fixed: 0007.txt
🔧 Fixed: 0008.txt
🔧 Fixed: 0009.txt
🔧 Fixed: 0011.txt
🔧 Fixed: 0019.txt
🔧 Fixed: 0020.txt
🔧 Fixed: 0021.txt
🔧 Fixed: 0023.txt
🔧 Fixed: 0026.txt
🔧 Fixed: 0028.txt
🔧 Fixed: 0029.txt
🔧 Fixed: 0031.txt
🔧 Fixed: 0034.txt
🔧 Fixed: 0035.txt
🔧 Fixed: 0036.txt
🔧 Fixed: 0037.txt
🔧 Fixed: 0039.txt
🔧 Fixed: 0040.txt
🔧 Fixed: 0042.txt
🔧 Fixed: 0043.txt
🔧 Fixed: 0048.txt
🔧 Fixed: 0051.txt
🔧 Fixed: 0053.txt
🔧 Fixed: 0054.txt
🔧 Fixed: 0056.txt
🔧 Fixed: 0058.txt
🔧 Fixed: 0061.txt
🔧 Fixed: 0062.txt
🔧 Fixed: 0063.txt
🔧 Fixed: 0067.txt
🔧 Fixed: 0068.txt
🔧 Fixed: 0069.txt
🔧 Fixed: 0072.txt
🔧 Fixed: 0073.txt
🔧 Fixed: 0074.txt
🔧 Fixed: 0075.txt
🔧 Fixed: 0076.txt
🔧 Fixed: 0080.txt
🔧 Fixed: 0081.txt
🔧 Fixed: 0088.txt
🔧 Fixed: 0089.txt
🔧 Fixed: 0091.txt
🔧 Fixed: 0092.txt
🔧 Fixed: 0094.txt
🔧 Fixed: 0095.txt
🔧 Fixed: 0098.txt
🔧 Fixed: 0100.txt
🔧 Fixed: 0101.txt
🔧 Fixed: 0104.txt
🔧 Fixed: 0109.txt
🔧 Fixed: 0111.txt
🔧 Fixed: 0

### Missing Label/Image Check

In [34]:
# Define image/label root folders
IMAGES_DIR = DATA_DIR / "images"
LABELS_DIR = DATA_DIR / "labels"

print("IMAGES_DIR:", IMAGES_DIR)
print("LABELS_DIR:", LABELS_DIR)


IMAGES_DIR: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images
LABELS_DIR: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\labels


In [35]:
train_img_stems = {p.stem for p in (IMAGES_DIR/"train").glob("*.jpg")}
train_lbl_stems = {p.stem for p in (LABELS_DIR/"train").glob("*.txt")}

missing_lbls = train_img_stems - train_lbl_stems
missing_imgs = train_lbl_stems - train_img_stems

print("Train Images:", len(train_img_stems))
print("Train Labels:", len(train_lbl_stems))
print("Missing Labels:", len(missing_lbls))
print("Missing Images:", len(missing_imgs))

Train Images: 1060
Train Labels: 1033
Missing Labels: 27
Missing Images: 0


### Train YOLOv8m

In [36]:
model = YOLO(str(MODELS_DIR / "weights" / "yolov8m.pt"))

results = model.train(
    data=str(yaml_path),
    project=str(RUNS_DIR),
    epochs=100,
    imgsz=640,
    batch=4,
    device=0,

    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.01,
    cos_lr=True,

    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
    erasing=0.0,

    workers=2,
    deterministic=True,
    name="yolov8m_stable",
)

New https://pypi.org/project/ultralytics/8.3.248 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.233  Python-3.11.0 torch-2.10.0.dev20251112+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\mast.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=D:\Sem3\CS581B_Robot_Percep

### Evaluate and Save Metrics

In [37]:
# Run validation
metrics = model.val(data=str(yaml_path), imgsz=640, project=str(RUNS_DIR), name ='val')

# Extract actual dictionary from metrics
metrics_dict = metrics.results_dict

print(metrics_dict)

# Save as CSV
import pandas as pd
# Ensure reports directory exists
REPORTS_DIR = BASE_DIR / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
metrics_path = REPORTS_DIR / "yolo_metrics.csv"

df = pd.DataFrame([metrics_dict])
df.to_csv(metrics_path, index=False)

print("✔ Metrics saved to reports/metrics.csv")


Ultralytics 8.3.233  Python-3.11.0 torch-2.10.0.dev20251112+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 229.456.9 MB/s, size: 15.3 KB)
val: Scanning D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\labels\val.cache... 121 images, 144 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 265/265 267.0Kit/s 0.0s
val: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0024.jpg: 1 duplicate labels removed
val: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0245.jpg: 1 duplicate labels removed
val: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0257.jpg: 1 duplicate labels removed
val: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0442.jpg: 1 duplicate labels removed
val: D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0694.jpg: 1 dupl

### Visuualize Predictions

In [38]:
import matplotlib.pyplot as plt
import random
import cv2

sample_imgs = random.sample(list(VAL_DIR.glob("*.jpg")), 5)

for img_path in sample_imgs:
    result = model(img_path)[0]
    img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(7,7))
    plt.imshow(result.plot()[:, :, ::-1])
    plt.title(img_path.name)
    plt.axis("off")
    plt.show()



image 1/1 D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0631.jpg: 480x640 2 obstacles, 10.8ms
Speed: 1.1ms preprocess, 10.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 700x700 with 1 Axes>


image 1/1 D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0391.jpg: 480x640 2 obstacles, 9.0ms
Speed: 0.9ms preprocess, 9.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 700x700 with 1 Axes>


image 1/1 D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0826.jpg: 480x640 2 obstacles, 8.2ms
Speed: 1.1ms preprocess, 8.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 700x700 with 1 Axes>


image 1/1 D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0275.jpg: 480x640 2 obstacles, 8.3ms
Speed: 0.9ms preprocess, 8.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 700x700 with 1 Axes>


image 1/1 D:\Sem3\CS581B_Robot_Perception\Project\Implementation\data\images\val\0232.jpg: 480x640 2 obstacles, 8.2ms
Speed: 0.9ms preprocess, 8.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 700x700 with 1 Axes>